### AJUSTE HIPERPARAMETROS PROCESO GAUSSIANO DE REGRESION PARA PREDECIR COEFICIENTE DE ACTIVIDAD (GPR)

### Cargar los datos ya limpiados y analizados en el Notebook analisis data y cargar librerias necesarias

In [4]:
!pip install -q -U keras-tuner

In [1]:
import numpy as np
import pandas as pd
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
datos=pd.read_csv("Data_clean_Completo4.csv")



In [2]:

variables_categoricas = ["Anion","Cation"]
variables_numericas = ["Solute_Mol_Mass", "Num_Cations", "Num_Anions", "Temperature_C", "Solvent_Mol_Mass", "Molal"]
variables_a_predecir = ["Molal_Activity_Coefficient", "Osmotic_Coefficient"]
column_transformer = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), variables_categoricas),
         ("num", StandardScaler(), variables_numericas)
    ])
X = column_transformer.fit_transform(datos[variables_categoricas + variables_numericas])
y = datos[["Molal_Activity_Coefficient", "Osmotic_Coefficient", "Solvent_Activity"]].values

In [3]:

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(datos, test_size=0.25, random_state=500, stratify=datos['Id_solucion'])
X_test = column_transformer.transform(test_data[variables_categoricas + variables_numericas])
y_test = test_data[variables_a_predecir].values
X_train = column_transformer.transform(train_data[variables_categoricas + variables_numericas])
y_train = train_data[variables_a_predecir].values
X_train=X_train.toarray()
X_test=X_test.toarray()
print(X_train.shape)
print(y_train.shape)


(6452, 167)
(6452, 2)


In [7]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


def build_model():
    kernel = ConstantKernel(1.0, (0.1, 10.0)) * Matern()
    gpr = GaussianProcessRegressor(kernel=kernel)
    model = Pipeline([('gpr', gpr)])
    return model


param_distributions = {
    'gpr__kernel__k2__length_scale': uniform(0, 1),
    'gpr__kernel__k2__nu': uniform(1, 1),
    'gpr__alpha': [0.1, 0.01, 0.001],
    'gpr__n_restarts_optimizer': np.arange(2, 11),
}


random_search = RandomizedSearchCV(
    build_model(),
    param_distributions,
    n_iter=10,
    scoring='neg_mean_absolute_percentage_error',
    n_jobs=-1,
    cv=5,
    refit=True,
    random_state=42,
)


random_search.fit(X_train, y_train[:, 0])



KeyboardInterrupt: 

In [ ]:
# Obtener mejores hiperparámetros y mejor puntuación
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Mejores hiperparámetros:", best_params)
print("Mejor puntuación (MAPE):", -best_score)